# Modèles 

In [4]:
import pandas as pd

Dans cette partie on va essayer d'utiliser des modèles d'apprentissage machine pour prédire les scores des joueurs. On commence par récupérer le dataset et le mettre en forme pour l'apprentissage.

Les choix que l'on fait sont les suivants :
  - On ne garde pas les variables qui dépendent du match :
      - list_var_avg + WIN
  - On ne garde pas les variables qui nous donne aucune information : 
      - SEASON_ID, GAME_ID
  - On modifi le type de la variable PLAYER_ID pour pas qu'il soit concidéré comme un nombre
  - On ne garde pas les variables agrégées sur les équipes car elles sont trop souvent `Nan` car les équipes ne jouent que rarement entre elles.

In [5]:
# La liste des variables dont on va récupérer les moyennes
list_var_avg = ['PTS', 'REB', 'AST', 'STL', 'BLK', 'FGM', 'FTM', 'FG3M',
                 'TOV', 'FGF', 'FG3F', 'FTF', 'BONUS', 'MALUS', 'SCORE']

In [8]:
# Récupere le dataset
df = pd.read_csv('../data/boxscore_2019_final.csv')

df = df.drop(list_var_avg[:-1]+['WIN', 'SEASON_ID', 'GAME_ID'], axis=1)
df = df[~df.AVG_SCORE.isna()]

df.PLAYER_ID = df.PLAYER_ID.astype(object)

# Pour l'instant on retire aussi les variables de moyenne par équipe car elles 
# sont trop souvant vide
df = df.drop(['AVG_ADD_'+x for x in list_var_avg]+['AVG_BONUS', 'AVG_MALUS', 'AVG_SCORE'], axis=1)

In [9]:
df.sample(3)

,PLAYER_ID,GAME_DATE,TEAM,ADV,HOME,SCORE,AVG_PTS,AVG_REB,AVG_AST,AVG_STL,AVG_BLK,AVG_FGM,AVG_FTM,AVG_FG3M,AVG_TOV,AVG_FGF,AVG_FG3F,AVG_FTF,NB_VIC,NB_VIC_ADD
10630,203992,2020-02-20,SAC,MEM,True,9,9.000000,3.666667,1.666667,0.333333,0.333333,3.333333,1.000000,1.333333,1.000000,8.000000,6.333333,0.000000,0,1
4866,202694,2020-02-28,LAC,DEN,True,4,12.333333,2.666667,2.000000,0.333333,1.000000,4.666667,1.000000,2.000000,2.666667,6.666667,3.000000,0.333333,2,0
16418,1628389,2019-11-12,MIA,DET,True,39,10.666667,10.000000,4.333333,2.666667,1.333333,4.000000,2.666667,0.000000,2.000000,4.333333,0.333333,3.666667,0,0


## 1 - Modèles linéaires

In [ ]:
# On ne garde que les joueurs qui ont joué plus de 50 matchs sur la saison
df_players = (df.groupby('PLAYER_ID').count() > 50)
list_players = df_players[df_players.SCORE].index
df = df[df.PLAYER_ID.isin(list_players)].reset_index(drop=True)